# Estimating Labor Market Returns to Education

In this exercise, we're going to use data from the [American Communities Survey (ACS)](https://usa.ipums.org/usa/acs.shtml) to study the relationship betwen educational attainment and wages. The ACS is a survey conducted by the United States Census Bureau (though it is not "The Census," which is a counting of every person in the United States that takes place every 10 years) to measure numerous features of the US population. The data we will be working with includes about 100 variables from the 2017 ACS survey, and is a 10% sample of the ACS (which itself is a 1% sample of the US population, so we're working with about a 0.1% sample of the United States). 

This data comes from [IPUMS](https://usa.ipums.org/usa/), which provides a very useful tool for getting subsets of major survey datasets, not just from the US, but [from government statistical agencies the world over](https://international.ipums.org/international-action/sample_details).

This is *real* data, meaning that you are being provided the data as it is provided by IPUMS. Documentation for all variables used in this data can be found [here](https://usa.ipums.org/usa-action/variables/group) (you can either search by variable name to figure out the meaning of a variable in this data, or search for something you want to see if a variable with the right name is in this data). 

Within this data is information on both the educational background and current earnings of a representative sample of Americans. We will now use this data to estimate the labor-market returns to graduating high school and college, and to learn something about the meaning of an educational degree. 

## Gradescope Autograding

Please follow [all standard guidance](https://www.practicaldatascience.org/ids720_specific/autograder_guidelines.html) for submitting this assignment to the Gradescope autograder, including storing your solutions in a dictionary called `results` and ensuring your notebook runs from the start to completion without any errors.

For this assignment, please name your file `exercise_dataframes.ipynb` before uploading.

You can check that you have answers for all questions in your `results` dictionary with this code:

```python
assert set(results.keys()) == {
    "ex2_num_obs",
    "ex3_num_vars",
    "ex8_updated_num_obs",
    "ex9_updated_num_obs",
    "ex11_grade12_income",
    "ex12_college_income",
    "ex12_college_income_pct",
    "ex14_high_school_dropout",
    "ex15_grade_9",
    "ex15_grade_10",
    "ex15_grade_11",
    "ex15_grade_12",
    "ex15_4_years_of_college",
    "ex15_graduate",
}
```

### Submission Limits

Please remember that you are **only allowed three submissions to the autograder.** Your last submission (if you submit 3 or fewer times), or your third submission (if you submit more than 3 times) will determine your grade Submissions that error out will **not** count against this total.


## Exercises

### Exercise 1

Data for these [exercises can be found here](https://github.com/nickeubank/MIDS_Data/tree/master/US_AmericanCommunitySurvey). 

Import `US_ACS_2017_10pct_sample.dta` into a pandas DataFrame (read it directly from a URL to help the autograder, please). 

This can be done with the command `pd.read_stata`, which will read in files created in the program Stata (and which uses the file suffix `.dta`). This is a format commonly used by social scientists.

In [1]:
import pandas as pd

results = {}

acs_data = pd.read_stata(
    "https://github.com/nickeubank/MIDS_Data/raw/refs/heads/master/US_AmericanCommunitySurvey/US_ACS_2017_10pct_sample.dta"
)

## Getting to Know Your Data

When you get a new dataset like this, it's good to start by trying to get a feel for its contents and organization. Toy datasets you sometimes get in classes are often very small, and easy to look at, but this is a pretty large dataset, so you can't just open it up and get a good sense of it. Here are some ways to get to know your data. 

### Exercise 2

How many observations are in your data? Store the answer in your `results` dictionary with the key `"ex2_num_obs"`.

In [2]:
ex2_num_obs = len(acs_data)
results["ex2_num_obs"] = ex2_num_obs
print(f"The number of observations in the data is {ex2_num_obs:,}")

The number of observations in the data is 319,004


### Exercise 3

How many variables are in your data? Store the answer in your `results` dictionary with the key `"ex3_num_vars"`.

In [3]:
ex3_num_vars = len(acs_data.columns)
results["ex3_num_vars"] = ex3_num_vars
print(f"The number of variables in the data is {ex3_num_vars}")

The number of variables in the data is 104


### Exercise 4

 Let's see what variables are in this dataset. First, try to see them all using the command:


```python
acs.columns
```

In [4]:
acs_data.columns

Index(['year', 'datanum', 'serial', 'cbserial', 'numprec', 'subsamp', 'hhwt',
       'hhtype', 'cluster', 'adjust',
       ...
       'migcounty1', 'migmet131', 'vetdisab', 'diffrem', 'diffphys', 'diffmob',
       'diffcare', 'diffsens', 'diffeye', 'diffhear'],
      dtype='object', length=104)

As you will see, `python` doesn't like to print out all the different variables when there are this many in a dataset. 

To get everything printed out, we can loop over all the columns and print them one at a time with the command:

```
for c in acs.columns: print(c)
```

It's definitely a bit of a hack, but honestly a pretty useful one!

In [5]:
for c in acs_data.columns:
    print(c, end=", ")

year, datanum, serial, cbserial, numprec, subsamp, hhwt, hhtype, cluster, adjust, cpi99, region, stateicp, statefip, countyicp, countyfip, metro, city, citypop, strata, gq, farm, ownershp, ownershpd, mortgage, mortgag2, mortamt1, mortamt2, respmode, pernum, cbpernum, perwt, slwt, famunit, sex, age, marst, birthyr, race, raced, hispan, hispand, bpl, bpld, citizen, yrnatur, yrimmig, language, languaged, speakeng, hcovany, hcovpriv, hinsemp, hinspur, hinstri, hcovpub, hinscaid, hinscare, hinsva, hinsihs, school, educ, educd, gradeatt, gradeattd, schltype, degfield, degfieldd, degfield2, degfield2d, empstat, empstatd, labforce, occ, ind, classwkr, classwkrd, looking, availble, inctot, ftotinc, incwage, incbus00, incss, incwelfr, incinvst, incretir, incsupp, incother, incearn, poverty, migrate1, migrate1d, migplac1, migcounty1, migmet131, vetdisab, diffrem, diffphys, diffmob, diffcare, diffsens, diffeye, diffhear, 

### Exercise 5

That's a *lot* of variables, and definitely more than we need. In general, life is easier when working with these kinds of huge datasets if you can narrow down the number of variables a little. In this exercise, we will be looking at the relationship between education and wages, we need variables for: 

- Age
- Income
- Education
- Employment status (is the person actually working)

These quantities of interest correspond to the following variables in our data: `age`, `inctot`, `educ`, and `empstat`. 

Subset your data to just those variables. 

In [6]:
acs_data_sub = acs_data[["age", "inctot", "educ", "empstat"]]
acs_data_sub

,age,inctot,educ,empstat
0,4,9999999,nursery school to grade 4,n/a
1,17,6000,grade 11,employed
2,63,6150,4 years of college,employed
3,66,14000,grade 12,not in labor force
4,1,9999999,n/a or no schooling,n/a
...,...,...,...,...
318999,33,22130,4 years of college,employed
319000,4,9999999,nursery school to grade 4,n/a
319001,20,5000,grade 12,employed
319002,47,240000,5+ years of college,employed


### Exercise 6 

Now that we have a more manageable number of variables, it's often very useful to look at a handful of rows of your data. The easiest way to do this is probably the `.head()` method (which will show you the first five rows), or the `tail()` method, which will show you the last five rows. 

But to get a good sense of your data, it's often better to use the `sample()` command, which returns a random set of rows. As the first and last rows are sometimes not representative, a random set of rows can be very helpful. Try looking at a random sample of 20 rows (note: you don't have to run `.sample()` ten times to get ten rows. Look at the `.sample` help file if you're stuck. 

In [7]:
acs_data_sub.sample(20)

,age,inctot,educ,empstat
115323,57,12000,grade 12,employed
282217,48,9500,grade 12,unemployed
64923,44,86000,4 years of college,employed
15175,8,9999999,nursery school to grade 4,n/a
275216,93,44840,grade 12,not in labor force
249309,19,1000,1 year of college,not in labor force
151824,32,3000,1 year of college,not in labor force
182421,46,26000,2 years of college,not in labor force
139010,4,9999999,nursery school to grade 4,n/a
212119,57,8000,grade 12,employed


### Exercise 7

Do you see any immediate problems? What issues do you see? (Please do answer in markdown)

I see 9999999 which doesn't look like a valid income having n/a employment status.

### Exercise 8 

One problem is that many people seem to have incomes of $9,999,999. Moreover, people with those incomes seem to be very young children. 

What you are seeing is one method (a relatively old one) for representing missing data. In this case, the value 9999999 is being used as a **sentinel value** — a way to denote missing data that was used back in the day when there was no way to add a special data type for mossing data. In this case, it identifies observations where the person is too young to work, so their income value is missing. 

So let's begin by dropping anyone who has `inctot` equal to 9999999.

After dropping, how many observations do you have? Save your answer in your `results` dictionary under the key `"ex8_updated_num_obs"`

In [8]:
acs_data_sentinel_dropped = acs_data_sub[acs_data_sub["inctot"] != 9999999]
ex8_updated_num_obs = len(acs_data_sentinel_dropped)
results["ex8_updated_num_obs"] = ex8_updated_num_obs
print(
    f"The number of observations after dropping the sentinel values is {ex8_updated_num_obs:,}"
)

The number of observations after dropping the sentinel values is 265,103


### Exercise 9

OK, the other potential problem is that our data includes lots of people who are unemployed and people who are not in the labor force (this means they not only don't have a job, but also aren't looking for a job). For this analysis, we want to focus on the wages of people who are currently employed. So subset the dataset for the people for whom `empstat` is equal to "employed". 

Note that our decision to only look at people who are employed impacts how we should interpret the relationship we estimate between education and income. Because we are only looking at employed people, we will be estimating the relationship between education and income *for people who are employed*. That means that if education affects the *likelihood* someone is employed, we won't capture that in this analysis.

(You might also want to run `.sample()` after this just to make sure you were successful in your subsetting).

After this subsetting, how many observations do you have? Save your answer in your `results` dictionary under the key `"ex9_updated_num_obs"`

In [9]:
acs_data_employed = acs_data_sentinel_dropped[
    acs_data_sentinel_dropped["empstat"] == "employed"
]
acs_data_employed.sample(20)

,age,inctot,educ,empstat
239566,54,30000,2 years of college,employed
136114,48,65000,2 years of college,employed
272547,21,11300,1 year of college,employed
41108,52,34000,4 years of college,employed
156478,45,150000,5+ years of college,employed
306000,34,26400,grade 12,employed
117532,33,11000,4 years of college,employed
313608,27,25000,2 years of college,employed
252016,64,87000,4 years of college,employed
91278,32,11440,4 years of college,employed


In [10]:
ex9_updated_num_obs = len(acs_data_employed)
results["ex9_updated_num_obs"] = ex9_updated_num_obs
print(
    f"The number of observations after subsetting for only employed status is {ex9_updated_num_obs:,}"
)

The number of observations after subsetting for only employed status is 148,758


### Exercise 10

Now let's turn to education. The `educ` variable seems to have a lot of discrete values. Let's see what values exist, and their distribution, using the `value_counts()` method. This is an *extremely* useful tool you'll use a lot! Try the following code (modified for the name of your dataset, of course):

```python
acs["educ"].value_counts()
```

In [11]:
acs_data_employed["educ"].value_counts()

educ
grade 12                     47815
4 years of college           33174
1 year of college            22899
5+ years of college          20995
2 years of college           14077
grade 11                      2747
grade 5, 6, 7, or 8           2092
grade 10                      1910
n/a or no schooling           1291
grade 9                       1290
nursery school to grade 4      468
Name: count, dtype: int64

### Exercise 11

There are a lot of values in here, so let's just check a couple. What is the average value of `inctot` for people whose highest grade level is "grade 12" (in the US, that is someone who has graduated high school)?

Save your answer in your `results` dictionary under the key `"ex11_grade12_income"`.

In [12]:
acs_data_employed_grade_12 = acs_data_employed[acs_data_employed["educ"] == "grade 12"]
ex11_grade12_income = acs_data_employed_grade_12["inctot"].mean()
results["ex11_grade12_income"] = ex11_grade12_income
print(
    f"The average income for people whose highest grade level is 'grade 12' is ${ex11_grade12_income:,.2f}"
)

The average income for people whose highest grade level is 'grade 12' is $38,957.76


### Exercise 12

What is the average income of someone who has completed an undergraduate degree but not done any postgraduate education ("4 years of college")? 

Save your answer in your `results` dictionary under the key `"ex12_college_income"`.

In percentage terms, how much does an employed college graduate earn as compared to someone who is only a high school graduate? Use the reference category that gives an answer above 100.

Store your answer in `"ex12_college_income_pct"`. Put your answer in percentage terms (so 100 implies they earn the same amount).

*Make sure to interpret your result in words when you print it out!*

In [13]:
acs_data_employed_4_yrs_college = acs_data_employed[
    acs_data_employed["educ"] == "4 years of college"
]
ex12_college_income = acs_data_employed_4_yrs_college["inctot"].mean()
results["ex12_college_income"] = ex12_college_income
print(
    f"The average income for people who have completed an undergraduate degree but not done any postgraduate education is ${ex12_college_income:,.2f}"
)

The average income for people who have completed an undergraduate degree but not done any postgraduate education is $75,485.05


In [14]:
ex12_college_income_pct = (
    results["ex12_college_income"] / results["ex11_grade12_income"] * 100
)
results["ex12_college_income_pct"] = ex12_college_income_pct
print(
    f"The percentage of income by employed college graduate as compared to someone who is only a high school graduate is {ex12_college_income_pct:,.2f}%"
)

The percentage of income by employed college graduate as compared to someone who is only a high school graduate is 193.76%



### Exercise 13
What does that suggest is the value of getting a college degree after graduating high school?

It suggests that with a college education, one can earn almost double what he/she can earn without it, making college education very valuable.

### Exercise 14

What is the average income for someone who has not finished high school? What does that suggest is the value of a high school diploma? (Treat `n/a or no schooling` as having no formal schooling, not as missing).

**Hint:** You may find the [.isin()](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.isin.html) method to be really helpful here.

Save your answer in your `results` dictionary under the key `"ex14_high_school_dropout"`.

In [15]:
acs_data_employed_school_dropout = acs_data_employed[
    acs_data_employed["educ"].isin(
        [
            "grade 11",
            "grade 5, 6, 7, or 8",
            "grade 10",
            "n/a or no schooling",
            "grade 9",
            "nursery school to grade 4",
        ]
    )
]
ex14_high_school_dropout = acs_data_employed_school_dropout["inctot"].mean()
results["ex14_high_school_dropout"] = ex14_high_school_dropout
print(
    f"The average income for people who's hightest eduction is below high school is ${ex14_high_school_dropout:,.2f}"
)

The average income for people who's hightest eduction is below high school is $26,226.46


In [16]:
percent_income_grade12_dropout = (
    results["ex11_grade12_income"] / results["ex14_high_school_dropout"] * 100
)
print(
    f"The percentage of income by high school leavers as compared to school dropouts is {percent_income_grade12_dropout:,.2f}%"
)

The percentage of income by high school leavers as compared to school dropouts is 148.54%


The result of 148.54% suggests that high school education is worth approximately 50% more in income than the income when high school is not attained.

### Exercise 15 

Complete the following table (storing values under the provided keys where listed):

- Average income for someone who only completed 9th grade (`ex15_grade_9`): _________
- Average income for someone who only completed 10th grade (`ex15_grade_10`): _________
- Average income for someone who only completed 11th grade (`ex15_grade_11`): _________
- Average income for someone who finished high school (12th grade) but never started college (`ex15_grade_12`): _________
- Average income for someone who completed 4 year of college (in the US, this corresponds to getting an undergraduate degree), but has no post-graduate education (no more than 4 years, `ex15_4_years_of_college`): _________
- Average income for someone who has some graduate education (more than 4 years, `ex15_graduate`): _________

In [17]:
results["ex15_grade_9"] = ex15_grade_9 = acs_data_employed[
    acs_data_employed["educ"] == "grade 9"
]["inctot"].mean()
results["ex15_grade_10"] = ex15_grade_10 = acs_data_employed[
    acs_data_employed["educ"] == "grade 10"
]["inctot"].mean()
results["ex15_grade_11"] = ex15_grade_11 = acs_data_employed[
    acs_data_employed["educ"] == "grade 11"
]["inctot"].mean()
results["ex15_grade_12"] = ex15_grade_12 = acs_data_employed[
    acs_data_employed["educ"] == "grade 12"
]["inctot"].mean()
results["ex15_4_years_of_college"] = ex15_4_years_of_college = acs_data_employed[
    acs_data_employed["educ"] == "4 years of college"
]["inctot"].mean()
results["ex15_graduate"] = ex15_graduate = acs_data_employed[
    acs_data_employed["educ"] == "5+ years of college"
]["inctot"].mean()

print(
    f"- Average income for someone who only completed 9th grade: ${ex15_grade_9:,.2f}"
)
print(
    f"- Average income for someone who only completed 10th grade: ${ex15_grade_10:,.2f}"
)
print(
    f"- Average income for someone who only completed 11th grade: ${ex15_grade_11:,.2f}"
)
print(
    f"- Average income for someone who finished high school (12th grade) but never started college: ${ex15_grade_12:,.2f}"
)
print(
    f"- Average income for someone who completed 4 year of college but has no post-graduate education: ${ex15_4_years_of_college:,.2f}"
)
print(
    f"- Average income for someone who has some graduate education (more than 4 years): ${ex15_graduate:,.2f}"
)

- Average income for someone who only completed 9th grade: $27,171.91
- Average income for someone who only completed 10th grade: $23,018.80
- Average income for someone who only completed 11th grade: $21,541.69
- Average income for someone who finished high school (12th grade) but never started college: $38,957.76
- Average income for someone who completed 4 year of college but has no post-graduate education: $75,485.05
- Average income for someone who has some graduate education (more than 4 years): $110,013.22


### Exercise 16 

Why do you think there is no benefit from moving from grade 9 to grade 10, or grade 10 to grade 11, but there is a huge benefit to moving from grade 11 to graduating high school (grade 12)?

(Think carefully before reading ahead!)

I think this is because incomes brackets are given by educational milestones achieved and high school is the first significant level. If a milestone or level isn't completed, one falls in the lower bracket. There's really no educational income bracket below high school.

In [18]:
# Testing
assert set(results.keys()) == {
    "ex2_num_obs",
    "ex3_num_vars",
    "ex8_updated_num_obs",
    "ex9_updated_num_obs",
    "ex11_grade12_income",
    "ex12_college_income",
    "ex12_college_income_pct",
    "ex14_high_school_dropout",
    "ex15_grade_9",
    "ex15_grade_10",
    "ex15_grade_11",
    "ex15_grade_12",
    "ex15_4_years_of_college",
    "ex15_graduate",
}

## Take-aways

Congratulations! You just discovered "the sheepskin effect!": people with degrees tend to earn substantially more than people who have *almost* as much education, but don't have an actual degree. 

In economics, this is viewed as evidence that the reason employers pay people with high school degrees more than those without degree is *not* that they think those who graduated high school have learned specific, useful skills. If that were the case, we would expect employee earnings to rise with every year of high school, since in each year of high school we learn more. 

Instead, this suggests employees pay high school graduates more because they think *the kind of people* who can finish high school are the *kind of people* who are likely to succeed at their jobs. Finishing high school, in other words, isn't about accumulating specific knowledge; it's about showing that you *are the kind of person* who can rise to the challenge of finishing high school, also suggesting you are also the kind of person who can succeed as an employee. 

(Obviously, this does not tell us whether that is an *accurate* inference, just that that seems to be how employeers think.) 

In other words, in the eyes of employers, a high school degree is a *signal* about the kind of person you are, not certification that you've learned a specific set of skills (an idea that earned [Michael Spence](https://en.wikipedia.org/wiki/Michael_Spence) a Nobel Prize in Economics). 